<a href="https://colab.research.google.com/github/freddyduitama/GVD/blob/master/0_7_PCA_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# instala el ambiente de spark..solo se corre una vez
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.2/spark-2.4.2-bin-hadoop2.7.tgz
!tar xf spark-2.4.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#Configura variables de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.2-bin-hadoop2.7"

In [0]:
#importa package
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import numpy as np

# librerias necesarias
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

In [0]:
# define  la sesion SPARK. 
conf = SparkConf().setAppName("ejemplo").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [0]:
#Crea la sesion
spark = SparkSession.builder.master("local[*]").getOrCreate()

# PCA


In [5]:
data =  [(Vectors.sparse(5, [(0, 1.0), (1,1.0), (2, 1.0)]),),
    (Vectors.dense([3.0, 3.0, 3.0, 0.0, 0.0]),),
    (Vectors.dense([4.0, 4.0, 4.0, 0.0, 0.0]),),
    (Vectors.dense([5.0, 5.0, 5.0, 0.0, 0.0]),),
    (Vectors.dense([0.0, 2.0, 0.0, 4.0, 4.0]),),
    (Vectors.dense([0.0, 0.0, 0.0, 5.0, 5.0]),),
    (Vectors.dense([0.0, 1.0, 0.0, 2.0, 2.0]),)
]

df = spark.createDataFrame(data, ["features"])

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

+-----------------------------------------+
|pcaFeatures                              |
+-----------------------------------------+
|[-1.31668353532568,-1.1240485367741655]  |
|[-3.9500506059770397,-3.3721456103224963]|
|[-5.26673414130272,-4.496194147096662]   |
|[-6.5834176766284,-5.620242683870827]    |
|[2.9000282007507683,-5.120649609103372]  |
|[4.559263125839789,-5.369852115245065]   |
|[1.4500141003753841,-2.560324804551686]  |
+-----------------------------------------+



#SVD

In [0]:
from pyspark import SparkContext
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

In [0]:
rows = sc.parallelize([
    Vectors.dense(1.0, 1.0, 1.0, 0.0, 0.0),
    Vectors.dense(3.0, 3.0, 3.0, 0.0, 0.0),
    Vectors.dense(4.0, 4.0, 4.0, 0.0, 0.0),
    Vectors.dense(5.0, 5.0, 5.0, 0.0, 0.0),
    Vectors.dense(0.0, 2.0, 0.0, 4.0, 4.0),
    Vectors.dense(0.0, 0.0, 0.0, 5.0, 5.0),
    Vectors.dense(0.0, 1.0, 0.0, 2.0, 2.0)
])

mat = RowMatrix(rows)

    # Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(2, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.

In [9]:
collected = U.rows.collect()
print("U factor is:")
for vector in collected:
     print(vector)
print("Singular values are: %s" % s)
print("V factor is:\n%s" % V)

U factor is:
[-0.13759912585743228,0.023611451432165555]
[-0.41279737757229684,0.07083435429649666]
[-0.5503965034297291,0.09444580572866222]
[-0.6879956292871613,0.11805725716082777]
[-0.15277508653386743,-0.5911009632535149]
[-0.07221651400482049,-0.7313118565123562]
[-0.07638754326693371,-0.29555048162675746]
Singular values are: [12.481014693580397,9.508614056636775]
V factor is:
DenseMatrix([[-0.56225841,  0.12664138],
             [-0.5928599 , -0.02877058],
             [-0.56225841,  0.12664138],
             [-0.09013354, -0.69537622],
             [-0.09013354, -0.69537622]])
